In [1]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
import numpy as np

import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

## Data Loading

In [3]:
import helper
data = helper.load_RWHAR(sel_location = "chest")
train_iter, val_iter,train_weight = helper.get_dataloaders(data, batch_size = 128, output_size = 8, val_pc = 0.3)

Cleaning and loading subject files


FileNotFoundError: [Errno 2] No such file or directory: './data/RWHAR/'

In [ ]:
from DeepConvLSTM_model import DeepConvNet
from Validation_model import Net
net = DeepConvNet(in_channels = 6,input_size = 50, hidden_size = 512, output_size = 8, conv_filter = (3,5), conv_padding = (1,2))
model = Net(model = net, num_classes = 8, classes_weight = torch.tensor(train_weight, dtype = torch.float), lr = 0.001)

trainer = pl.Trainer(gpus=-1,
                     max_epochs=100,
                    #  limit_train_batches=0.1,
                    #  limit_val_batches=0.1,
                     log_every_n_steps = 200,
                     callbacks = [EarlyStopping(monitor = "val_f1_score", patience = 5, mode = "max")],
                     logger = TensorBoardLogger(save_dir = 'Validation_LSTM_logs/', name = "RWHAR"),
                     stochastic_weight_avg=True
                     )
trainer.fit(model, train_iter, val_iter)

In [2]:
from TransformerClassifier import TransformerClassifier
from Validation_model import Net
net = TransformerClassifier(in_channels = 6, output_size = 8, d_model = 50, nhead = 5, dim_feedforward = 10000, num_layers = 5)
model = Net(model = net, num_classes = 8, classes_weight = torch.tensor(train_weight, dtype = torch.float), lr = 0.01)

trainer = pl.Trainer(gpus=-1,
                     max_epochs=100,
                    #  limit_train_batches=0.1,
                    #  limit_val_batches=0.1,
                     log_every_n_steps = 200,
                     callbacks = [EarlyStopping(monitor = "val_f1_score", patience = 7, mode = "max")],
                     logger = TensorBoardLogger(save_dir = 'Validation_transformer_logs/', name = "RWHAR"),
                     stochastic_weight_avg=True
                     )
trainer.fit(model, train_iter, val_iter)

NameError: name 'train_weight' is not defined

## Saved model

### Transformer Classifier

Version : 3

location : Validation_transformer_logs/RWHAR/version_3/checkpoints/epoch=66-step=14471.ckpt

F1 score : 88.7%

loss : 0.3502

in_channels = 6, output_size = 8, d_model = 50, nhead = 5, dim_feedforward = 10000, num_layers = 5

### LSTM model

version : 1

location : Validation_LSTM_logs/RWHAR/version_1/checkpoints/epoch=14-step=3239.ckpt

F1 score : 86.22%

loss : 0.3768

in_channels = 6,input_size = 50, hidden_size = 512, output_size = 8, conv_filter = (3,5), conv_padding = (1,2)
